### RelTune : Relation-Aware Bayesian Optimization of DBMS Configurations Guided by Affinity Scores

### Relational Graph Construction
- Parameter Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
import itertools

import json

file_path = "/Description_Embeding_fixed.json"
with open(file_path, "r") as f:
    data = json.load(f)
    

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
param_names = list(data.keys())
descriptions = list(data.values())
embeddings = model.encode(descriptions, convert_to_tensor=True)
desc_embeddings = model.encode(descriptions, convert_to_tensor=True)

results = []
for i, j in itertools.combinations(range(len(param_names)), 2):
    param_i = param_names[i]
    param_j = param_names[j]
    sim = F.cosine_similarity(embeddings[i].unsqueeze(0), embeddings[j].unsqueeze(0)).item()
    results.append((param_i, param_j, sim))
    
results.sort(key=lambda x: -x[2])


In [2]:
import torch

edge_list = []
similarity_threshold = 0.75

for i in range(len(embeddings)):
    for j in range(i + 1, len(embeddings)):
        sim = F.cosine_similarity(embeddings[i], embeddings[j], dim=0).item()
        if sim >= similarity_threshold:
            edge_list.append((i, j))
            edge_list.append((j, i))  


edge_index = torch.tensor(edge_list, dtype=torch.long).T  


In [ ]:
from torch_geometric.data import Data


x = embeddings  
graph_data = Data(x=x, edge_index=edge_index)


In [6]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx


G = to_networkx(graph_data, to_undirected=True)


param_name_dict = {i: name for i, name in enumerate(param_names)}
G = nx.relabel_nodes(G, param_name_dict)


In [ ]:
plt.figure(figsize=(16, 16))
pos = nx.spring_layout(G, seed=42, k=0.5)  


nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=500)
nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_labels(G, pos, font_size=9)

plt.title("MySQL Parameter Graph (Cosine ≥ 0.75)", fontsize=14)
plt.axis("off")
plt.show()


- Graph Construction

In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

config_file = "configs.csv"
config = pd.read_csv(config_file)

scaler = MinMaxScaler()
scaled_config = scaler.fit_transform(config)  # shape: [5000, 138]

metrics_df = pd.read_csv("metrics.csv")
metric_scaler = MinMaxScaler()
scaled_metrics = metric_scaler.fit_transform(metrics_df[['tps', 'latency']])

In [9]:
graph_dataset = []
for i in range(len(scaled_config)):
    feature_list = []
    for j, param in enumerate(param_names):
        value_tensor = torch.tensor([scaled_config[i][j]], dtype=torch.float)  
        desc_tensor = desc_embeddings[j]  
        node_feature = torch.cat([value_tensor, desc_tensor], dim=0)  
        feature_list.append(node_feature)
    x = torch.stack(feature_list, dim=0)  
    y = torch.tensor(scaled_metrics[i], dtype=torch.float).view(1,-1)
    
    data = Data(x=x, edge_index=edge_index, y=y)

    graph_dataset.append(data)

In [14]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

metrics = pd.read_csv("external_metrics.csv")

metrics = metrics.drop(['Unnamed: 0'], axis = 1)

metrics = metrics.replace([np.inf],9999999)


In [ ]:
knob_list = glob.glob("configs/my_*.cnf")

cnt = 0

for xx in range(len(knob_list)):
    path = "configs/my_{}.cnf".format(xx)
    # knob_list = glob.glob("/home/sein/2023_EDBT/KCC_tpcc_dataset/my_*.cnf")
    a_all = pd.read_csv(path, sep="=", names=['Sample', 'value'], header=2)
    a_all = a_all.set_index("Sample")
    cur_all_df = a_all.T
    
    if cnt == 0:
        configs = cur_all_df
    else :
        configs = pd.concat([configs, cur_all_df], axis=0)
    cnt += 1
configs = configs.reset_index()
configs = configs.drop(["index"],axis=1)
configs = configs.drop(configs.columns[[0,1]], axis=1)


configs

### Graph Based Latent Representation

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool

class GATEncoder(nn.Module):
    def __init__(self, in_dim=385, hidden_dim=128, hidden_sec_dim=64, z_dim=32, heads=4):
        super().__init__()
        self.gat1 = GATConv(in_dim, hidden_dim // heads, heads=heads)
        self.gat2 = GATConv(hidden_dim, hidden_sec_dim // heads, heads=heads)
        self.gat3 = GATConv(hidden_sec_dim, z_dim, heads=1, concat=False)  # 마지막은 concat=False로 z_dim 유지

    def forward(self, x, edge_index, batch):
        x = F.elu(self.gat1(x, edge_index))
        x = F.elu(self.gat2(x, edge_index))
        x = self.gat3(x, edge_index)
        z = global_mean_pool(x, batch)
        return z  # [batch_size, z_dim]

class ConfigDecoder(nn.Module):
    def __init__(self, z_dim=32, output_dim=138):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, z):
        return self.fc(z)

class Surrogate(nn.Module):
    def __init__(self, z_dim=32):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
            # nn.Sigmoid()
        )

    def forward(self, z):
        return self.fc(z)

class GNN_Autoencoder_With_Surrogate(nn.Module):
    def __init__(self, in_dim=385, hidden_dim=128, hidden_sec_dim=64, z_dim=32, out_config_dim=138, heads=4):
        super().__init__()
        self.encoder = GATEncoder(in_dim, hidden_dim, hidden_sec_dim, z_dim, heads=heads)
        self.decoder = ConfigDecoder(z_dim, out_config_dim)
        self.surrogate = Surrogate(z_dim)

    def forward(self, data):
        z = self.encoder(data.x, data.edge_index, data.batch)
        recon_config = self.decoder(z)
        metric_pred = self.surrogate(z)
        return recon_config, metric_pred, z


In [ ]:
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNN_Autoencoder_With_Surrogate().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
loss_fn = nn.MSELoss()
metric_weight = 0.3  

train_data, temp_data = train_test_split(graph_dataset, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.3, random_state=42)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

best_val_loss = float('inf')
patience = 50
counter = 0

for epoch in range(1, 3001):
    model.train()
    total_recon_loss, total_metric_loss, total_combined_loss = 0, 0, 0

    for batch in train_loader:
        batch = batch.to(device)
        recon_config, metric_pred, _ = model(batch)

        target_config = batch.x[:, 0].view(batch.num_graphs, -1)          
        metric_target = batch.y.view(batch.num_graphs, -1)                

        recon_loss = loss_fn(recon_config, target_config)
        metric_loss = loss_fn(metric_pred, metric_target)
        loss = recon_loss + metric_weight * metric_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_recon_loss += recon_loss.item()
        total_metric_loss += metric_loss.item()
        total_combined_loss += loss.item()

    avg_recon_loss = total_recon_loss / len(train_loader)
    avg_metric_loss = total_metric_loss / len(train_loader)
    avg_train_loss = total_combined_loss / len(train_loader)

    model.eval()
    val_recon_loss, val_metric_loss, val_combined_loss = 0, 0, 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            recon_config, metric_pred, _ = model(batch)

            target_config = batch.x[:, 0].view(batch.num_graphs, -1)
            metric_target = batch.y.view(batch.num_graphs, -1)

            recon_loss = loss_fn(recon_config, target_config)
            metric_loss = loss_fn(metric_pred, metric_target)
            loss = recon_loss + metric_weight * metric_loss

            val_recon_loss += recon_loss.item()
            val_metric_loss += metric_loss.item()
            val_combined_loss += loss.item()

    avg_val_recon = val_recon_loss / len(val_loader)
    avg_val_metric = val_metric_loss / len(val_loader)
    avg_val_loss = val_combined_loss / len(val_loader)

    if epoch % 50 == 0:
        print(f"[Epoch {epoch}]")
        print(f"  Train Recon Loss:  {avg_recon_loss:.4f} | Metric Loss: {avg_metric_loss:.4f} | Total: {avg_train_loss:.4f}")
        print(f"  Val   Recon Loss:  {avg_val_recon:.4f} | Metric Loss: {avg_val_metric:.4f} | Total: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            print(f"Best Validation Loss: {best_val_loss:.4f}")
            break

In [21]:
from torch_geometric.loader import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(graph_dataset, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.3, random_state=42)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)
test_loader = DataLoader(test_data, batch_size=64)

### Latent Space Optimization via Hybrid-Score-Guided Bayesian Optimization

In [ ]:

import torch
import numpy as np
from skopt import gp_minimize
from skopt.space import Real

aff_score_list = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
z_dim = 'hyper-parameter'
alpha = 'hyper-parameter'   
gamma = 'hyper-parameter'   
sigma = 'hyper-parameter'   


z_list = []
y_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        _, _, z = model(batch)
        z_list.append(z.cpu())
        y_list.append(batch.y.cpu())

Z_all = torch.cat(z_list, dim=0).numpy()  
Y_all = torch.cat(y_list, dim=0).numpy()  


def is_good(metric, tps_threshold='hyper-parameter', latency_threshold='hyper-parameter'):

    return metric[0] >= tps_threshold and metric[1] <= latency_threshold

Z_good = np.array([z for z, m in zip(Z_all, Y_all) if is_good(m)])


def affinity_score(z, Z_good, sigma=1.0):
    if len(Z_good) == 0:
        return 0.0
    dists = np.linalg.norm(Z_good - z, axis=1)
    weights = np.exp(-dists**2 / (2 * sigma**2))
    return np.mean(weights)


z_min = Z_all.min(axis=0)
z_max = Z_all.max(axis=0)
margin = 'hyper-parameter'
z_range = z_max - z_min
z_min -= margin * z_range
z_max += margin * z_range

space = [Real(float(z_min[i]), float(z_max[i]), name=f'z_{i}') for i in range(z_dim)]

def objective(z_flat):
    z_tensor = torch.tensor(z_flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        metric = model.surrogate(z_tensor)
        tps = metric[:, 0].item()
        latency = metric[:, 1].item()
        score = tps - alpha * latency

    aff = affinity_score(np.array(z_flat), Z_good, sigma=sigma)
    hybrid_score = score + gamma * aff

    ### 그림그리기 용
    aff_score_list.append(hybrid_score)
    
    return -hybrid_score  

res = gp_minimize(
    func=objective,
    dimensions=space,
    n_calls= 300,
    n_initial_points=10,
    acq_func="EI",
    random_state=42
)

aff_z_trajectory = np.array(res.x_iters)  
np.save("z_trajectory_hybrid.npy", aff_z_trajectory)

best_z = torch.tensor(res.x, dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    best_metric = model.surrogate(best_z)
    best_config = model.decoder(best_z)

print("\nHybrid Score)")
print(f"Best Score (TPS - α·Latency + γ·Affinity): {-res.fun:.4f}")
print(f"TPS: {best_metric[0,0].item():.4f}, Latency: {best_metric[0,1].item():.4f}")
print("Recovered Config (normalized):")
print(best_config)


In [36]:
import glob
import os
import sys
import pandas as pd

knob_list = glob.glob("configs/my_*.cnf")


In [ ]:
cnt = 0

for xx in range(len(knob_list)):
    path = "configs/my_{}.cnf".format(xx)
    a_all = pd.read_csv(path, sep="=", names=['Sample', 'value'], header=2)
    a_all = a_all.set_index("Sample")
    cur_all_df = a_all.T
    
    if cnt == 0:
        A_config = cur_all_df
    else :
        A_config = pd.concat([A_config, cur_all_df], axis=0)
    cnt += 1
A_config = A_config.reset_index()
A_config = A_config.drop(["index"],axis=1)
A_config = A_config.drop(A_config.columns[[0,1]], axis=1)


A_config

In [ ]:

recovered = best_config.cpu().detach().numpy()  
original_config = scaler.inverse_transform(recovered.reshape(1, -1)).flatten()

for name, value in zip(param_names, original_config):
        print(f"{name} = {int(round(value))}")


In [ ]:
import torch
import numpy as np
from skopt import gp_minimize
from skopt.space import Real

vbo_score_list = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
z_dim = 'hyper-parameter'
alpha = 'hyper-parameter'

z_list, y_list = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        _, _, z = model(batch)
        z_list.append(z.cpu())
        y_list.append(batch.y.cpu())
Z_all = torch.cat(z_list, dim=0).numpy()
Y_all = torch.cat(y_list, dim=0).numpy()

# 2. 탐색 공간 정의
z_min = Z_all.min(axis=0)
z_max = Z_all.max(axis=0)
margin = 'hyper-parameter'
z_min -= margin * (z_max - z_min)
z_max += margin * (z_max - z_min)
space = [Real(float(z_min[i]), float(z_max[i]), name=f'z_{i}') for i in range(z_dim)]

def vanilla_objective(z_flat):
    z_tensor = torch.tensor(z_flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        metric = model.surrogate(z_tensor)
        tps = metric[:, 0].item()
        latency = metric[:, 1].item()
        score = tps - alpha * latency
    vbo_score_list.append(score)       
    return -score 

res = gp_minimize(
    func=vanilla_objective,
    dimensions=space,
    n_calls=300,
    n_initial_points=10,
    acq_func="EI",
    random_state=42
)

vbo_z_trajectory = np.array(res.x_iters)


best_z = torch.tensor(res.x, dtype=torch.float32).unsqueeze(0).to(device)
with torch.no_grad():
    best_metric = model.surrogate(best_z)
    best_config = model.decoder(best_z)

print("[Vanilla BO")
print(f"Best Score (TPS - α·Latency): {-res.fun:.4f}")
print(f"TPS: {best_metric[0,0].item():.4f}, Latency: {best_metric[0,1].item():.4f}")
print("Recovered Config (normalized):")
print(best_config)
